# Creating raw frame

In [1]:
import wmfdata as wmf
import pandas as pd

In [2]:
HIVE_SNAPSHOT = "2022-04"
END_OF_DATA = "2022-05-01 00:00:00"
DAYS_IN_LAST_3_MO = 89 # Feb, Mar, Apr 2022

In [3]:
# Select all users who were active editors during 2 of the last 3 months.
frame_query = f"""
WITH monthly_edits AS (
  SELECT
    event_user_text AS user_name,
    YEAR(event_timestamp) AS year,
    MONTH(event_timestamp) AS month,
    SUM(IF(wiki_db = "wikidatawiki", 0.1, 1)) AS edits
  FROM wmf.mediawiki_history
  WHERE
    snapshot = "{HIVE_SNAPSHOT}" AND   
    -- REGISTERED
    NOT event_user_is_anonymous AND
    -- NON-BOT
    SIZE(event_user_is_bot_by) = 0 AND
    -- CONTENT
    page_namespace_is_content_historical AND
    -- EDITS
    event_entity = "revision" AND
    event_type = "create" AND
    -- FROM THE LAST 3 MONTHS
    UNIX_TIMESTAMP(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") 
        > (UNIX_TIMESTAMP("{END_OF_DATA}") - (60 * 60 * 24 * {DAYS_IN_LAST_3_MO}))
  -- PER USER, PER MONTH
  GROUP BY event_user_text, YEAR(event_timestamp), MONTH(event_timestamp)
), 
yearly_project_edits AS (
  SELECT
    event_user_text AS user_name,
    wiki_db AS project,
    sum(if(wiki_db = "wikidatawiki", 0.1, 1)) AS edits,
    max(event_timestamp) AS latest_edit
  FROM wmf.mediawiki_history
  WHERE
    snapshot = "{HIVE_SNAPSHOT}" AND
    -- REGISTERED
    NOT event_user_is_anonymous AND
    -- NON-BOT
    SIZE(event_user_is_bot_by) = 0 AND
    -- EDITS
    event_entity = "revision" AND
    event_type = "create" AND
    -- FROM THE LAST YEAR
    UNIX_TIMESTAMP(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") 
        > (UNIX_TIMESTAMP("{END_OF_DATA}") - (60 * 60 * 24 * 365))
  -- PER USER, PER WIKI
  GROUP BY event_user_text, wiki_db
)
SELECT 
  recent_actives.user_name AS user_name,
  yearly_edits.project AS home_project,
  edits AS global_edits
from (
    -- USERS ACTIVE IN 2 OF 3 MONTHS
    SELECT
      user_name,
      SUM(CAST(edits >= 5 AS INT)) as active_months
    FROM monthly_edits
    WHERE
      -- NOT WMF STAFF
      user_name NOT LIKE "%WMF%"
    GROUP BY user_name
    HAVING active_months >= 2
) recent_actives
LEFT JOIN (
  SELECT
    user_name,
    project,
    -- in the unlikely event that wikis are tied by edit count and latest edit, 
    -- row_number() will break it somehow
    ROW_NUMBER() OVER (PARTITION BY user_name ORDER BY edits DESC, latest_edit DESC) AS rank,
    SUM(edits) OVER (PARTITION BY user_name) AS edits
  FROM yearly_project_edits
) yearly_edits
ON 
  recent_actives.user_name = yearly_edits.user_name AND
  rank = 1
"""

raw_frame = wmf.spark.run(frame_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [4]:
frame = (
  raw_frame
  .set_index("user_name")
  .assign(
    global_edits=lambda df: df["global_edits"].astype("float"),
    home_project=lambda df: df["home_project"].astype("category")
  )
)

# The `reset_index` prevents the first and last usernames from appearing in the output
frame.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59907 entries, 0 to 59906
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_name     59907 non-null  object  
 1   home_project  59907 non-null  category
 2   global_edits  59907 non-null  float64 
dtypes: category(1), float64(1), object(1)
memory usage: 1.0+ MB


## Limiting to emailable users and adding emails

In [5]:
def make_sql_tuple(i):
    """
    Making an SQL 'tuple', for use in an IN clause, is hard. Doing it manually using 
    `", ".join` requires a lot of messing around with quote marks and escaping. Using the
    string representation of a Python tuple *almost* works, but fails when there's just
    one element, because SQL doesn't accept the trailing comma that Python uses.

    What we really want is the string representation of a Python list, but using parentheses
    instead of brackets. This function turns an iterable into just that.
    """    

    if type(i) != list:
        i = [x for x in i]
    
    list_repr = repr(i)
    
    return "(" + list_repr[1:-1] + ")"

In [6]:
# Query centralauth.globaluser table in replicas to obtain email addresses
# This seems to be more reliable that the user tables of individual wikis
# since the global email address is what's shown to the user on every
# local preferences page; if the address is changed on an individual wiki, 
# the global address and that wiki's address will be immediately updated but the
# local addresses at other wikis seem not to be.
#
# We also want confirmed email addresses only, since we don't want to email
# people whose addresses have been used without their consent.
users = make_sql_tuple(frame.index)

email_query = f"""
SELECT 
  gu_name AS user_name,
  gu_email AS email
FROM centralauth.globaluser
WHERE
  gu_name IN {users} AND
  gu_email_authenticated IS NOT NULL AND
  gu_email != ""
"""

user_emails = wmf.mariadb.run(
  email_query,
  dbs="centralauth"
).set_index("user_name")

In [7]:
# The left join will exclude users without confirmed email addresses.
frame = pd.merge(user_emails, frame, on="user_name", how="left")
frame.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36673 entries, 0 to 36672
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_name     36673 non-null  object  
 1   email         36673 non-null  object  
 2   home_project  36673 non-null  category
 3   global_edits  36673 non-null  float64 
dtypes: category(1), float64(1), object(2)
memory usage: 950.9+ KB


In [8]:
# Remove any users who have the `disablemail` preference set
projects = frame["home_project"].unique()
f = []

for project in projects:
  users = make_sql_tuple(frame.query("home_project == @project").index)
  
  props = wmf.mariadb.run(f"""
  SELECT
    user_name
  FROM user
  LEFT JOIN user_properties
  ON user_id = up_user
  WHERE
    user_name in {users} AND
    up_property = "disablemail" AND
    up_value = 1
  """, dbs=project)
  
  f.append(props)

disablemail_users = pd.concat(f)["user_name"]

In [9]:
frame = frame.drop(disablemail_users)
frame.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33355 entries, 0 to 33354
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_name     33355 non-null  object  
 1   email         33355 non-null  object  
 2   home_project  33355 non-null  category
 3   global_edits  33355 non-null  float64 
dtypes: category(1), float64(1), object(2)
memory usage: 866.7+ KB


# Removing users with duplicate emails

In [10]:
duplicate_email_users = frame[frame["email"].duplicated(keep=False)]
(
  duplicate_email_users
  .groupby("email")
  .apply(len)
  .value_counts()
  .pipe(pd.DataFrame)
  .rename({0: "number of users"}, axis="columns")
  .rename_axis(index="number of emails")
)

,number of users
number of emails,
2,115
3,4
5,2


In [11]:
duplicate_users = (
  duplicate_email_users
  # Among accounts sharing an email, treat the one with the most edits as primary 
  .sort_values("global_edits", ascending=False)
  # Include only the non-primary accounts as true duplicates
  [lambda df: df["email"].duplicated(keep="first")]
)

frame = frame.drop(duplicate_users.index)
frame.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33224 entries, 0 to 33223
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_name     33224 non-null  object  
 1   email         33224 non-null  object  
 2   home_project  33224 non-null  category
 3   global_edits  33224 non-null  float64 
dtypes: category(1), float64(1), object(2)
memory usage: 863.4+ KB


# Remove opt-outs

In [12]:
# This list needs to be obtained from Global Data and Insights
optouts = pd.read_table("secrets/optouts.tsv", squeeze=True)
frame = frame.drop(optouts, errors="ignore")
frame.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33224 entries, 0 to 33223
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_name     33224 non-null  object  
 1   email         33224 non-null  object  
 2   home_project  33224 non-null  category
 3   global_edits  33224 non-null  float64 
dtypes: category(1), float64(1), object(2)
memory usage: 863.4+ KB


In [13]:
# optouts by username
optouts_username = pd.read_csv('secrets/optouts_username.csv', header=None)
optouts_username.columns = ['user_name']
optouts_username.set_index('user_name', inplace=True)
frame = frame[~frame.index.isin(optouts_username.index)]

# Divide into strata

In [14]:
# Add project groups
grouped_projects = (
  pd.read_table("definitions/project-group-assignments.tsv")
  .drop("project_name", axis = 1)
  .assign(
    project_key=lambda df: df["project_key"].astype("category"),
    project_group=lambda df: df["project_group"].astype("category")
  )
)

frame = (
  frame
  .reset_index()
  .merge(grouped_projects, how = "left", left_on = "home_project", right_on = "project_key" )
  .drop("project_key", axis = 1)
  .assign(home_project=lambda df: df["home_project"].astype("category"))
  .set_index("user_name")
)

# Any project without an explicit assignment is in the "other" group
frame["project_group"] = frame["project_group"].fillna("other")

In [15]:
# Add edit bins
bin_edges = [10, 30, 150, 600, 1200, float("inf")]
frame["edit_bin"] = pd.cut(
  frame["global_edits"], 
  bin_edges,
  right=False,
  labels=["10-29", "30-149", "150-599", "600-1199", "1200+"]
)

In [16]:
frame.groupby(["project_group", "edit_bin"]).size().unstack()

edit_bin,10-29,30-149,150-599,600-1199,1200+
project_group,,,,,
arwiki,25,62,59,29,97
asia_wps,8,51,77,49,129
cee_wps,115,476,594,302,945
commons,117,440,515,306,1483
dewiki,80,505,700,336,850
enwiki,537,2884,2632,1078,2810
eswiki,80,312,369,151,361
frwiki,115,520,559,237,639
itwiki,40,227,255,103,324


In [17]:
frame.groupby(["project_group", "edit_bin"]).size().unstack().to_csv('definitions/strata-populations.tsv', sep='\t')

# Save frame

In [18]:
# Note: This file now contains senstive information.
frame.to_parquet("secrets/sampling-frame.parquet", index=True)